# Propuesta de preguntas

## BLOQUE 1 · Evolución general por género (base del estudio)

¿Cómo evolucionó la popularidad media de cada género (pop, rock, clásica y reguetón) en España entre 2019 y 2024?

¿Qué géneros ganaron o perdieron popularidad durante la cuarentena frente al periodo pre-COVID?

¿Se recuperaron los géneros tras la pandemia o se produjo un cambio estructural en las preferencias musicales post-COVID?

## BLOQUE 2 · Impacto del COVID (parte “storytelling”)

¿Qué cambios en el ranking de canciones más populares se produjeron durante la cuarentena respecto a los años anteriores?

¿Aparecieron nuevos artistas relevantes durante la cuarentena que no tenían presencia previa en el periodo pre-COVID?

¿La pandemia favoreció un mayor consumo de música más rítmica (reguetón/pop) frente a géneros tradicionales como rock o clásica?

## BLOQUE 3 · Artistas y concentración de popularidad

¿Se concentró más la popularidad en unos pocos artistas durante la cuarentena o se diversificó el consumo musical?

¿Qué artistas mostraron un mayor crecimiento de popularidad entre el periodo pre-COVID y el post-COVID?

¿Cambió la estabilidad de los artistas top (top 10 / top 20) antes, durante y después de la pandemia?

## BLOQUE 4 · Análisis por género

¿Qué género mostró una mayor rotación de canciones populares (más canciones nuevas por año) durante la pandemia?

¿La música clásica experimentó patrones de consumo distintos al resto de géneros durante el confinamiento?

¿El reguetón mantuvo su crecimiento de popularidad tras el COVID o alcanzó un punto de saturación?

## BLOQUE 5 · Spotify vs Last.fm

¿Existen diferencias entre la popularidad medida por Spotify y la popularidad medida por Last.fm para los mismos artistas?

¿Qué artistas tienen alta presencia en Spotify pero menor engagement en Last.fm, y viceversa?

## BLOQUE 6 · Comparativa temporal clara

¿Qué canciones o artistas fueron representativos de cada etapa: pre-COVID, COVID y post-COVID?

¿Qué géneros mostraron una recuperación más rápida tras el fin de las restricciones?


### Las 10 más equilibradas para un proyecto (recomendación)

Evolución de la popularidad por género (2019–2024)

Cambios de popularidad durante la cuarentena

Recuperación post-COVID por género

Aparición de nuevos artistas durante la pandemia

Concentración de popularidad en artistas top

Crecimiento de artistas pre vs post-COVID

Rotación de canciones populares por género

Diferencias de comportamiento entre géneros

Comparación Spotify vs Last.fm

Artistas/canciones representativas de cada etapa

In [5]:
# Para hacer pausas (útil si hacemos muchas peticiones seguidas)
import time                                 
import requests
# Para type hints (ayuda a entender qué devuelve cada función)
from typing import Any, Dict, Optional

BASE_URL = "https://ws.audioscrobbler.com/2.0/"
API_KEY = '3bd1dc669ba0fd4ee507954de7dd51a0'

1) Creamos una función reutilizable para llamar a last.fm de forma estándar.

2) Construimos un diccionario con parámetros obligatorios:

  - `method`, `api_key`, `format`

3) Añadimos parámetros específicos (`artist`, `autocorrect`, etc.) usando `**params`.

4) Comprobamos errores:

  - Errores HTTP con `raise_for_status()´

  - Errores “internos” del JSON de last.fm con `if "error" in data`

In [6]:
def lastfm_request(method: str, params: Dict[str, Any], api_key: str = API_KEY, timeout: int = 30) -> Dict[str, Any]:
    """
    Función genérica para llamar a la API de Last.fm y devolver el JSON como diccionario.
    
    - method: nombre del método/endpoint de last.fm (ej: 'artist.getInfo')
    - params: parámetros específicos del método (ej: {'artist': 'Coldplay'})
    - api_key: la clave de la API
    - timeout: segundos máximos que esperamos antes de cancelar la petición
    """

    # Construimos todos los parámetros que necesita last.fm en una sola variable
    full_params = {
        "method": method,     # Qué método vamos a usar (ej: artist.getInfo)
        "api_key": api_key,   # Nuestra API Key
        "format": "json",     # Queremos la respuesta en formato JSON
        **params              # Aquí metemos el resto de parámetros específicos (artist, autocorrect, etc.)
    }

    # Hacemos la petición GET a la URL base, pasando los parámetros
    r = requests.get(BASE_URL, params=full_params, timeout=timeout)

    # Si el servidor responde con error HTTP (404, 401, 500...), aquí salta una excepción
    r.raise_for_status()

    # Convertimos la respuesta a JSON (en Python: dict/list)
    data = r.json()

    # A veces last.fm devuelve error "dentro" del JSON aunque el HTTP sea 200 (OK)
    if isinstance(data, dict) and "error" in data:
        raise ValueError(f"Last.fm API error {data.get('error')}: {data.get('message')}")

    # Si todo va bien, devolvemos el JSON
    return data


In [7]:
# Llamamos al método artist.getInfo para pedir información del artista "Coldplay"
data = lastfm_request(
    "artist.getInfo",
    {"artist": "Coldplay", "autocorrect": 1}  # autocorrect=1 corrige nombres mal escritos
)

# Vemos qué claves (keys) tiene el JSON principal de respuesta
data.keys()

dict_keys(['artist'])

In [8]:
# Extraemos el bloque "artist" del JSON (si no existe, usamos {} para evitar errores)
artist = data.get("artist", {})

# Vemos qué información trae dentro el bloque del artista
artist.keys()

dict_keys(['name', 'mbid', 'url', 'image', 'streamable', 'ontour', 'stats', 'similar', 'tags', 'bio'])

In [9]:
# Extraemos la parte "artist" del JSON
# Usamos {} como valor por defecto por si no existiera la clave
artist = data.get("artist", {})

# Mostramos todas las claves que contiene el bloque del artista
artist.keys()

dict_keys(['name', 'mbid', 'url', 'image', 'streamable', 'ontour', 'stats', 'similar', 'tags', 'bio'])

## ¿Qué nos devuelve la API aquí?

`artist` es un diccionario con toda la información del artista.

Las claves más importantes para el proyecto son:

`name`: nombre del artista

`stats`: estadísticas de popularidad

`bio`: biografía del artista

`similar`: artistas similares

Otras claves (`image`, `ontour`, `streamable`) existen pero no son relevantes para nuestro análisis.

📌 Con esto confirmamos que last.fm nos da más información a nivel artista que a nivel canción, lo que refuerza nuestra decisión de usar el artista como entidad principal.

In [10]:
# Obtenemos las estadísticas del artista (oyentes y reproducciones)
artist.get("stats", {}), artist.get("bio", {}).keys()

({'listeners': '8929500', 'playcount': '763666459'},
 dict_keys(['links', 'published', 'summary', 'content']))

¿Qué información contiene cada bloque?

`stats`

 - `listeners`: número de oyentes únicos

 - `playcount`: número total de reproducciones

📌 Ambos valores vienen como strings, por lo que más adelante habrá que convertirlos a `int` para poder analizarlos en SQL.

`bio`

 - `summary`: resumen corto de la biografía

 - `content`: biografía completa

 - `published`: fecha de publicación de la biografía

📌 Esto nos permite hacer análisis cualitativos, no solo numéricos.

In [11]:
{
    # Nombre del artista
    "name": artist.get("name"),

    # Estadísticas de popularidad
    "listeners": artist.get("stats", {}).get("listeners"),
    "playcount": artist.get("stats", {}).get("playcount"),

    # Comprobamos si existen textos de biografía
    "bio_has_summary": "summary" in artist.get("bio", {}),
    "bio_has_content": "content" in artist.get("bio", {}),
}

{'name': 'Coldplay',
 'listeners': '8929500',
 'playcount': '763666459',
 'bio_has_summary': True,
 'bio_has_content': True}

¿Qué estamos haciendo aquí?

 - Estamos filtrando el JSON enorme de la API.

 - Nos quedamos solo con la información útil para el proyecto.

 - Comprobamos si la biografía existe (`True / False`) para evitar errores más adelante.

📌 Este paso es clave porque:

 - La API devuelve muchos datos.

 - Nosotras queremos estructurar la información pensando en la base de datos.

In [12]:
from typing import Any, Dict

def get_artist_info(artist_name: str, autocorrect: int = 1) -> Dict[str, Any]:
    """
    Pide a la API de Last.fm información de un artista y devuelve un diccionario "limpio"
    y siempre con las mismas claves (aunque falten datos).
    
    - artist_name: nombre del artista que queremos buscar
    - autocorrect: si vale 1, Last.fm intenta corregir nombres mal escritos
    """

    # 1) Llamamos al endpoint/método "artist.getInfo" de Last.fm
    #    Le pasamos como parámetros el nombre del artista y autocorrect
    data = lastfm_request(
        "artist.getInfo",
        {"artist": artist_name, "autocorrect": autocorrect}
    )

    # 2) Dentro del JSON, la info principal viene en la clave "artist"
    #    Usamos {} por defecto para evitar errores si no existiera
    artist = data.get("artist", {}) or {}

    # 3) Dentro de artist, las estadísticas están dentro de la clave "stats"
    stats = artist.get("stats", {}) or {}

    # 4) La biografía está dentro de la clave "bio"
    bio = artist.get("bio", {}) or {}

    # 5) Devolvemos un diccionario con los datos que nos interesan
    #    - Convertimos listeners y playcou

¿Qué hace esta función y por qué está así?

 - Llama a la API de Last.fm usando el método `artist.getInfo`.

 - La API devuelve un JSON grande, y nosotros lo transformamos en un diccionario pequeño y ordenado.

 - `listeners` y `playcount` vienen como texto (string), por eso los convertimos a `int` para poder analizarlos en SQL.

 - Usamos `.get(..., {})` y `or {}` para evitar errores si alguna parte no viene en la respuesta.

In [13]:
# Probamos la función con un artista conocido
# Si funciona, debe devolver un diccionario con:
# name, mbid, url, listeners, playcount y bio
get_artist_info("Coldplay")

¿Qué respuesta nos da la API aquí?

 - La función devuelve un diccionario ya “normalizado”.

 - Ejemplo de campos que vemos:

  - `artist_name`: nombre del artista (ya corregido si autocorrect estaba activado)

  - `listeners`: oyentes únicos

  - `playcount`: reproducciones totales

  - `bio_summary` y `bio_content`: biografía en texto (resumen y completa)

In [14]:
# Probamos con un nombre mal escrito para ver si last.fm lo corrige automáticamente
# autocorrect=1 intenta arreglar el nombre y devolver el artista correcto
get_artist_info("Colplay", autocorrect=1)

¿Qué estamos comprobando aquí?

 - Que aunque el usuario escriba mal el nombre del artista, la API puede corregirlo si activamos `autocorrect=1`.

Esto es útil para evitar errores y para hacer el sistema más “amigable” con entradas reales.

In [15]:
from typing import List

def get_similar_artists(artist_name: str, limit: int = 10, autocorrect: int = 1) -> List[Dict[str, Any]]:
    """
    Devuelve una lista de artistas similares para un artista dado.
    Cada elemento de la lista es un diccionario con info del artista similar.
    """
    data = lastfm_request(
        "artist.getSimilar",
        {
            "artist": artist_name,
            "limit": limit,
            "autocorrect": autocorrect
        }
    )

    # La lista suele venir dentro de: data["similarartists"]["artist"]
    similar_block = data.get("similarartists", {}) or {}
    similar_list = similar_block.get("artist", []) or []

    results = []

    # Recorremos cada artista similar y normalizamos lo que nos interesa
    for a in similar_list:
        results.append({
            "source_artist": artist_name,
            "similar_artist_name": a.get("name"),
            "similar_artist_mbid": a.get("mbid") or None,
            "similar_artist_url": a.get("url") or None,
            # "match" suele venir como string tipo "0.87" -> lo convertimos a float
            "match": float(a["match"]) if a.get("match") else None
        })

    return results


¿Qué nos devuelve la API y qué guardamos?

 - Este endpoint devuelve una lista de artistas “parecidos”.

 - Nosotros lo normalizamos en una lista de diccionarios con:

  - `source_artist`: artista original (el que estamos consultando)

  - `similar_artist_name`: nombre del artista similar

  - `match`: nivel de similitud (lo convertimos a float para análisis)

In [16]:
similar = get_similar_artists("Coldplay", limit=5)
similar

[{'source_artist': 'Coldplay',
  'similar_artist_name': 'Keane',
  'similar_artist_mbid': 'c7020c6d-cae9-4db3-92a7-e5c561cbad50',
  'similar_artist_url': 'https://www.last.fm/music/Keane',
  'match': 1.0},
 {'source_artist': 'Coldplay',
  'similar_artist_name': 'Imagine Dragons',
  'similar_artist_mbid': '012151a8-0f9a-44c9-997f-ebd68b5389f9',
  'similar_artist_url': 'https://www.last.fm/music/Imagine+Dragons',
  'match': 0.781353},
 {'source_artist': 'Coldplay',
  'similar_artist_name': 'OneRepublic',
  'similar_artist_mbid': 'c33c2065-b1c3-4406-b066-d33a9e2ea71a',
  'similar_artist_url': 'https://www.last.fm/music/OneRepublic',
  'match': 0.725816},
 {'source_artist': 'Coldplay',
  'similar_artist_name': 'Snow Patrol',
  'similar_artist_mbid': 'a66999a7-ae5c-460e-ba94-1a01143ae847',
  'similar_artist_url': 'https://www.last.fm/music/Snow+Patrol',
  'match': 0.628487},
 {'source_artist': 'Coldplay',
  'similar_artist_name': 'Travis',
  'similar_artist_mbid': '22a40b75-affc-4e69-8884-2

In [17]:
from typing import Any, Dict, List

def get_top_artists_by_genre(genre: str, limit: int = 50) -> List[Dict[str, Any]]:
    """
    Devuelve el top de artistas de un género (tag) usando la API de Last.fm.
    """
    data = lastfm_request(
        "tag.getTopArtists",
        {"tag": genre, "limit": limit}
    )

    artists = data.get("topartists", {}).get("artist", [])

    results = []

    for artist in artists:
        results.append({
            "genre": genre,
            "artist_name": artist.get("name"),
            "artist_mbid": artist.get("mbid") or None,
            "artist_url": artist.get("url") or None,
            "rank": int(artist.get("@attr", {}).get("rank")) if artist.get("@attr") else None
        })

    return results

In [18]:
top_rock = get_top_artists_by_genre('rock', limit=50)
top_rock[:3]

[{'genre': 'rock',
  'artist_name': 'Radiohead',
  'artist_mbid': 'a74b1b7f-71a5-4011-9441-d0b5e4122711',
  'artist_url': 'https://www.last.fm/music/Radiohead',
  'rank': 1},
 {'genre': 'rock',
  'artist_name': 'Paramore',
  'artist_mbid': '44cf61b8-5197-448a-b82b-cef6ee89fac5',
  'artist_url': 'https://www.last.fm/music/Paramore',
  'rank': 2},
 {'genre': 'rock',
  'artist_name': 'Coldplay',
  'artist_mbid': 'cc197bad-dc9c-440d-a5b5-d52ba2e14234',
  'artist_url': 'https://www.last.fm/music/Coldplay',
  'rank': 3}]

In [22]:
top_rock_lista = []

for artista in top_rock:
    top_rock_lista.append(artista['artist_name'])
    
top_rock_lista

['Radiohead',
 'Paramore',
 'Coldplay',
 'Linkin Park',
 'Red Hot Chili Peppers',
 'Fleetwood Mac',
 'My Chemical Romance',
 'Queen',
 'Maroon 5',
 'The Beatles',
 'The Cranberries',
 'The Police',
 'Foo Fighters',
 'Panic! at the Disco',
 'Harry Styles',
 'Weezer',
 'Nirvana',
 'Fall Out Boy',
 'David Bowie',
 'Oasis',
 'Goo Goo Dolls',
 'Jeff Buckley',
 'Evanescence',
 'Green Day',
 'Bruce Springsteen',
 'Måneskin',
 'U2',
 'The Rolling Stones',
 'Billy Idol',
 'Muse',
 'Nickelback',
 'Paul McCartney',
 'Three Days Grace',
 'Eagles',
 'John Lennon',
 'The Cardigans',
 'Bon Jovi',
 'Chris Isaak',
 'OneRepublic',
 'Dire Straits',
 'Toto',
 'Lenny Kravitz',
 'Charlie Brown Jr.',
 'The All-American Rejects',
 'R.E.M.',
 'Kings of Leon',
 'Crowded House',
 'Bryan Adams',
 'Creed',
 "Guns N' Roses"]

In [24]:
top_rock_lista_normalizada = []

for artista in top_rock_lista:
    normalizado = artista.lower().strip().casefold()
    top_rock_lista_normalizada.append(normalizado)

top_rock_lista_normalizada

['radiohead',
 'paramore',
 'coldplay',
 'linkin park',
 'red hot chili peppers',
 'fleetwood mac',
 'my chemical romance',
 'queen',
 'maroon 5',
 'the beatles',
 'the cranberries',
 'the police',
 'foo fighters',
 'panic! at the disco',
 'harry styles',
 'weezer',
 'nirvana',
 'fall out boy',
 'david bowie',
 'oasis',
 'goo goo dolls',
 'jeff buckley',
 'evanescence',
 'green day',
 'bruce springsteen',
 'måneskin',
 'u2',
 'the rolling stones',
 'billy idol',
 'muse',
 'nickelback',
 'paul mccartney',
 'three days grace',
 'eagles',
 'john lennon',
 'the cardigans',
 'bon jovi',
 'chris isaak',
 'onerepublic',
 'dire straits',
 'toto',
 'lenny kravitz',
 'charlie brown jr.',
 'the all-american rejects',
 'r.e.m.',
 'kings of leon',
 'crowded house',
 'bryan adams',
 'creed',
 "guns n' roses"]

In [20]:
top_rock_info_artist = []

for i in top_rock_lista:
    artista = get_artist_info(i)
    top_rock_info_artist.append(artista)
    
top_rock_info_artist[:5]

[None, None, None, None, None]

# Extracción 1000 artistas por género

In [1]:
import time
import csv
import requests

BASE_URL = "https://ws.audioscrobbler.com/2.0/"
API_KEY = "3bd1dc669ba0fd4ee507954de7dd51a0" 

In [2]:
def normalize(name: str) -> str:
    """strip + lower + casefold (tal y como lo necesitas)"""
    return name.strip().lower().casefold()

In [3]:
def get_top_artists_tag(tag: str, total: int = 1000, limit: int = 50, pause_s: float = 0.25, timeout: int = 30):
    """
    Devuelve una lista con 'total' nombres de artistas (normalizados) para un tag.
    Hace las páginas necesarias (1000 con limit=50 -> 20 peticiones).
    """
    pages = (total + limit - 1) // limit
    names = []
    seen = set()

    for page in range(1, pages + 1):
        params = {
            "method": "tag.getTopArtists",
            "api_key": API_KEY,
            "format": "json",
            "tag": tag,
            "limit": limit,
            "page": page
        }

        # Petición simple (si quieres retries/backoff, lo añadimos después)
        r = requests.get(BASE_URL, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()

        artists = data.get("topartists", {}).get("artist", [])
        if not artists:
            break

        for a in artists:
            n = normalize(a.get("name", ""))
            if n and n not in seen:
                seen.add(n)
                names.append(n)
                if len(names) >= total:
                    return names

        time.sleep(pause_s)

    return names


In [4]:
def save_csv(names, filepath: str):
    """Guarda 1 columna: artist_name"""
    with open(filepath, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["artist_name"])
        for n in names:
            w.writerow([n])

In [5]:
# --- Ejecutar y crear 4 CSV (uno por género) ---
genres = {
    "rock": "rock",
    "pop": "pop",
    "reggaeton": "reggaeton",
    "chill": "chill"
}

for genre, tag in genres.items():
    print(f"Descargando {genre}...")
    names = get_top_artists_tag(tag=tag, total=1000, limit=50)
    out = f"lastfm_{genre}_top1000.csv"
    save_csv(names, out)
    print(f"OK -> {out} ({len(names)} artistas)")

Descargando rock...
OK -> lastfm_rock_top1000.csv (1000 artistas)
Descargando pop...
OK -> lastfm_pop_top1000.csv (1000 artistas)
Descargando reggaeton...
OK -> lastfm_reggaeton_top1000.csv (1000 artistas)
Descargando chill...
OK -> lastfm_chill_top1000.csv (1000 artistas)
